In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [ ]:
# import sys
# import logging

# nblog = open("nb_freqWords.log", "a+")
# sys.stdout.echo = nblog
# sys.stderr.echo = nblog

# get_ipython().log.handlers[0].stream = nblog
# get_ipython().log.setLevel(logging.INFO)

# %autosave 5

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")
import pandas as pd
from tqdm import tqdm
import nltk
import re
import ast
from nltk.corpus import stopwords
from collections import Counter, defaultdict
nltk.download('stopwords')
import glob
from textCleaning import parse_stopwords, get_file_names, extract_year_ipc
import pandas as pd

c:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP\.conda\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edgar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP\.conda\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the c

In [61]:
import importlib
import textCleaning
importlib.reload(textCleaning)
from textCleaning import parse_stopwords, get_file_names, extract_year_ipc

c:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP\.conda\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edgar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/"
# path = "/home/edgarlanoue/data/"

In [13]:
technet = pd.read_csv(path+"Technet/clean_vocab.csv")
tn_lemm = pd.read_csv(path+"Technet/lemmatized_technet.csv")

In [14]:

file_path = path+"PatentNovelty/technet/stopwords.txt" 
sW_dict = parse_stopwords(file_path)

# Access each list by its label
stopwords_nltk = sW_dict.get("Stopwords NLTK", [])
stopwords_ustpo = sW_dict.get("Stopwords USTPO", [])
stopwords_tn_technical = sW_dict.get("Stopwords TN_technical", [])
to_add = sW_dict.get("toAdd", [])
new_sw = sW_dict.get("newSW", [])

# Print the lists to verify
print("length= ", len(stopwords_nltk), ", ", "Stopwords NLTK:", stopwords_nltk)
print("length= ", len(stopwords_ustpo), ", ", "Stopwords USTPO:", stopwords_ustpo)
print("length= ", len(stopwords_tn_technical), ", ", "Stopwords TN_technical:", stopwords_tn_technical)
print("length= ", len(to_add), ", ", "toAdd:", to_add)
print("length= ", len(new_sw), ", ", "newSW:", new_sw)

stopwords = list(set([item for sublist in sW_dict.values() for item in sublist]))

length=  179 ,  Stopwords NLTK: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only'

In [62]:
vocab=textCleaning.Vocab(technet=technet, df_lemm=tn_lemm, stopwords=stopwords)
# len(vocab.tn_lemm_filtered) # 845647

In [44]:
path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/"
pathOutput = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"

In [ ]:
def cleanCSV(path, pathOutput):
    tE_names = get_file_names(path+"toEval")
    KS_names = get_file_names(path+"KS")
    ES_names = get_file_names(path+"ES")

    tE_set = set([extract_year_ipc(string) for string in tE_names])
    KS_set = set([extract_year_ipc(string) for string in KS_names])
    ES_set = set([extract_year_ipc(string) for string in ES_names])

    assert(tE_set==KS_set==ES_set)

    yearList = [year for year, ipc in list(tE_set)]
    ipcList = [ipc for year, ipc in list(tE_set)]

    for year in yearList:
        for ipc in ipcList:
            print(f"{year}")
            print(f"    {ipc}")
            tE = pd.read_csv(path+f"exemple données/test/toEval/{year}_{ipc}_patents_toEval.csv")
            matching_files = glob.glob(path+f"exemple données/test/KS/{year}_*_{ipc}_KS_raw.csv")[0]
            KS = pd.read_csv(matching_files)
            matching_files = glob.glob(path+f"exemple données/test/ES/{year}_*_{ipc}_ES_raw.csv")[0]
            ES = pd.read_csv(matching_files)

            tE_clean = vocab.lemmDF(vocab.cleanDF(tE, type="all"))
            KS_clean = vocab.lemmDF(vocab.cleanDF(KS, type="all"))
            ES_clean = vocab.lemmDF(vocab.cleanDF(ES, type="all"))

            tE_clean.to_csv(path+f"exemple données/cleaned/tE/{year}_{ipc}_tE_cleaned.csv", index=False)
            KS_clean.to_csv(path+f"exemple données/cleaned/KS/{year}_{ipc}_KS_cleaned.csv", index=False)
            ES_clean.to_csv(path+f"exemple données/cleaned/ES/{year}_{ipc}_ES_cleaned.csv", index=False)

    return None

        

In [6]:
path_tE = path+"exemple données/test/toEval"
path_KS = path+"exemple données/test/KS"
path_ES = path+"exemple données/test/ES"

tE_names = get_file_names(path_tE)
KS_names = get_file_names(path_KS)
ES_names = get_file_names(path_ES)

tE_set = set([extract_year_ipc(string) for string in tE_names])
KS_set = set([extract_year_ipc(string) for string in KS_names])
ES_set = set([extract_year_ipc(string) for string in ES_names])

assert(tE_set==KS_set==ES_set)

yearList = [year for year, ipc in list(tE_set)]
ipcList = [ipc for year, ipc in list(tE_set)]

In [7]:
for year, ipc in zip(yearList, ipcList):
    print(f"{year}")
    print(f"    {ipc}")
    tE = pd.read_csv(path+f"exemple données/test/toEval/{year}_{ipc}_patents_toEval.csv")
    matching_files = glob.glob(path+f"exemple données/test/KS/{year}_*_{ipc}_KS_raw.csv")[0]
    KS = pd.read_csv(matching_files)
    matching_files = glob.glob(path+f"exemple données/test/ES/{year}_*_{ipc}_ES_raw.csv")[0]
    ES = pd.read_csv(matching_files)

2016
    H01L


In [63]:
tE_clean = vocab.lemmDF(vocab.cleanDF(tE, type="all"))

Clean


100%|██████████| 11700/11700 [00:15<00:00, 767.66it/s]


Lemmatize


100%|██████████| 11700/11700 [00:01<00:00, 7436.56it/s]


In [216]:
tE_clean = vocab.lemmDF(vocab.cleanDF(tE, type="all"))
KS_clean = vocab.lemmDF(vocab.cleanDF(KS, type="all"))
ES_clean = vocab.lemmDF(vocab.cleanDF(ES, type="all"))

Clean


100%|██████████| 11700/11700 [00:16<00:00, 689.54it/s]


Lemmatize


100%|██████████| 11700/11700 [00:01<00:00, 7072.62it/s]


Clean


...

100%|██████████| 99087/99087 [00:13<00:00, 7435.62it/s]


Clean


100%|██████████| 284544/284544 [09:09<00:00, 517.91it/s]


Lemmatize


100%|██████████| 284544/284544 [00:48<00:00, 5825.28it/s]


In [65]:
tE_clean.to_csv(path+f"exemple données/cleaned/tE/{year}_{ipc}_tE_cleaned.csv", index=False)

In [ ]:
tE_clean.to_csv(path+f"exemple données/cleaned/tE/{year}_{ipc}_tE_cleaned.csv", index=False)
KS_clean.to_csv(path+f"exemple données/cleaned/KS/{year}_{ipc}_KS_cleaned.csv", index=False)
ES_clean.to_csv(path+f"exemple données/cleaned/ES/{year}_{ipc}_ES_cleaned.csv", index=False)

In [211]:
# Vérification
# count_word_frequency_total(clean_KS_H01L_2016) - c'est bon